## Import libraries

In [2]:
import numpy as np
import tensorflow as tf

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2

## Importing data

In [3]:
data = pd.read_csv('data/data.csv')

In [5]:
# Let's check how many samples for each category are present
print("Total number of images in the dataset: ", len(data))

label_count = data['isDeepfake'].value_counts()
print(label_count)

Total number of images in the dataset:  5479
isDeepfake
True     3113
False    2366
Name: count, dtype: int64


In [ ]:
ORI_FRAMES_PATH = "data/frames/original2"
FAKE_FRAMES_PATH = "data/frames/faked2"

